# Tile-Based Math Game in Unity — Riassunto concettuale completo

Questo progetto **non è solo un esercizio di matematica**, ma una **pipeline 2D completa** che copre i fondamenti reali dello sviluppo in Unity:

* setup di progetto e scena
* fisica 2D
* animator e state machine
* player controller
* object pooling
* game loop
* UI guidata dai dati

È progettato per insegnare **struttura e architettura**, non solo funzionalità isolate.

Fonte: 

---

## 1. Unity 6 LTS: perché è una scelta consapevole

L’uso di **Unity 6 LTS** non è casuale.

Concetti chiave:

* LTS = comportamento stabile nel tempo
* meno regressioni e bug casuali
* stesso progetto riapribile anche dopo mesi o anni

Mentalità da produzione, non da tutorial temporaneo.

---

## 2. Setup 2D corretto (fondamenta del progetto)

Scelte fondamentali:

* Editor in modalità **2D**
* Camera **Orthographic**
* Aspect ratio fisso **16:9**

Motivazioni:

* gioco a camera fissa
* coordinate prevedibili
* UI allineabile in modo deterministico

Nei giochi tile-based la prevedibilità è più importante del realismo.

---

## 3. Tags, Layers e Sorting Layers (ordine e controllo)

### Sorting Layers

Ordine definito con precisione:

```
Jetpack
Player
Obstacle
Ship
UI
```

Perché è importante:

* controllo esplicito del rendering
* niente hack con Order in Layer estremi
* separazione chiara tra visual e logica

### Tag e Layer

* **Tag**: identificazione logica (Player, Floor, Obstacle)
* **Layer**: collisioni e filtri fisici

Tag = chi è l’oggetto
Layer = con chi interagisce

---

## 4. Gerarchia con ROOT (pattern fondamentale)

Ogni entità complessa è costruita con un **GameObject ROOT**:

```
Ship (ROOT)
 └─ Sprite
 └─ Colliders
 └─ Tubes
```

```
Ground (ROOT)
 └─ GroundTile
```

Perché usare un ROOT:

* trasformazioni globali sicure
* separazione tra visual e logica
* prefab più flessibili

È lo stesso pattern usato anche nei controller 3D e FPS.

---

## 5. Player 2D: fisica semplice ma corretta

Componenti principali:

* Rigidbody2D
* CapsuleCollider2D
* Rotazione Z bloccata

Motivazioni:

* movimento fisico coerente
* collisioni morbide
* prevenzione di rotazioni indesiderate

Anche in 2D la fisica va **controllata**, non lasciata al caso.

---

## 6. Animator come State Machine

Struttura:

* enum `PlayerState`
* parametro int `State` nell’Animator

Mappatura:

```
0 = Idle
1 = Walking
2 = Flying
3 = Stunned
```

Concetto chiave:

* lo stato governa l’animazione
* l’animazione non decide la logica

Disabilitare **Has Exit Time**:

* transizioni istantanee
* ideale per pixel art

È una Finite State Machine applicata correttamente.

---

## 7. PlayerController: organizzazione corretta

Il controller è suddiviso per responsabilità:

* `FixedUpdate()` → fisica
* `CheckInputs()` → input
* `SetState()` → decisione dello stato
* `Move()` / `Fly()` → azione
* `Stun()` → evento
* `IsGrounded()` → sensing

Questa struttura è chiara, leggibile e facilmente estendibile.

### Ground check con Raycast

* controllo preciso
* nessun rumore da collisioni
* indipendente dal movimento

---

## 8. Ostacoli: comportamento e lifecycle

Ogni ostacolo:

* si muove
* ruota
* si disattiva automaticamente dopo un tempo

```csharp
Invoke("Deactivate", aliveTime);
```

Motivazione:

* evitare `Destroy`
* preparare il terreno per il pooling

Introduzione corretta all’ottimizzazione.

---

## 9. Object Pooling (concetto chiave)

Il pool:

* istanzia gli oggetti all’avvio
* riutilizza gli stessi GameObject
* abilita e disabilita invece di creare/distruggere

Pattern centrale:

```csharp
pooledObjects.Find(x => !x.activeInHierarchy);
```

Vantaggi:

* prestazioni più stabili
* scalabilità
* pattern usato in progetti reali

Questo concetto è assolutamente da portfolio.

---

## 10. Spawner basato sulla camera

Lo spawner:

* calcola i bordi della camera
* genera oggetti a sinistra o destra
* assegna automaticamente la direzione

```csharp
float camWidth = (2 * orthographicSize) * aspect;
```

Perché è ben progettato:

* nessun valore hardcoded
* adattabile a qualsiasi risoluzione
* dipendente dalla camera, non dalla scena

---

## 11. Problem class: data-driven design

La classe `Problem`:

* non eredita da MonoBehaviour
* contiene solo dati
* è serializzabile

```csharp
[System.Serializable]
public class Problem
```

Motivazione:

* separazione tra dati e comportamento
* facile editing da Inspector
* riutilizzabile

È lo stesso principio dei ScriptableObject, in forma semplificata.

---

## 12. GameManager: gestione del game loop

Responsabilità:

* problema corrente
* timer
* condizioni di vittoria e sconfitta
* validazione delle risposte

Singleton semplice:

```csharp
public static GameManager instance;
```

Non è il pattern più avanzato, ma è:

* chiaro
* didattico
* sufficiente per il contesto

Qui si impara cos’è davvero un **game loop**.

---

## 13. UI guidata dai dati

La UI:

* non contiene logica di gioco
* visualizza solo lo stato corrente

Esempio:

```csharp
timeDial.fillAmount =
    remainingTime / timePerProblem;
```

Concetto chiave:

* normalizzazione (0–1)
* separazione UI / logica

È un approccio MVC semplificato.

---

## 14. ProblemTube: trigger e delega

Ogni tubo:

* conosce solo il proprio ID
* non conosce la risposta corretta
* delega al GameManager

```csharp
GameManager.instance.OnPlayerEnterTube(tubeId);
```

Vantaggi:

* basso accoppiamento
* logica centralizzata
* facilità di estensione

---

